In [1]:
import numpy as np

In [2]:
EPSILON = 10e-8
MAX_ITER = 1000

In [3]:
def generate_init_pt(A, b, c):
    if isinstance(x_0, np.ndarray):
        return x_0, z_0,pi_0
    AAT_inv = np.linalg.inv(A@A.T)
    pi_bar = AAT_inv@A@c
    z_bar = c - A.T@pi_bar
    x_bar = A.T@AAT_inv@b
    del_x = max(-1.5 * min(x_bar), 0)
    del_z = max(-1.5 * min(z_bar), 0)
    t = (x_bar + del_x*np.ones(len(x_bar))).T.dot(z_bar + del_z * np.ones(len(z_bar)))
    del_x_bar = del_x + (t/(2*sum(z_bar+del_z)))
    del_z_bar = del_z + (t/(2*sum(x_bar+del_x)))
    x0, z0, pi0 = x_bar + del_x_bar, z_bar + del_z_bar, pi_bar
    return x0, z0, pi0

In [4]:
def met_stopping_condition(Q, A, b, c, x, z, pi, n_iter):
    primal_feasible = np.linalg.norm(A@x-b) < EPSILON
    dual_feasible = np.linalg.norm(-Q.T@x + A.T@pi+z-c) < EPSILON
    zero_dual_gap = x.T@z < EPSILON
    return primal_feasible and dual_feasible and zero_dual_gap or n_iter > MAX_ITER

def get_alpha(Q, x, z, d_x, d_z, eta=1):
    problem_is_quadratic = np.count_nonzero(Q) > 0
    alpha_x_max = min(1, min((-x / d_x)[d_x<0]))
    alpha_z_max = min(1, min((-z / d_z)[d_z<0]))
    if problem_is_quadratic:
        alpha = min(1, eta*alpha_x_max, eta*alpha_z_max)
        return alpha, alpha   
    return min(alpha_x_max*eta, 1), min(alpha_z_max*eta, 1)

def get_d(M, F, num_vars, num_constraints):
    d = np.linalg.solve(M, F)
    d_x = d[:num_vars]
    d_z = d[-num_vars:]
    d_pi = d[num_vars:num_vars + num_constraints]

    return d_x, d_z, d_pi

In [5]:
def min_func(Q, A, b, c, verbose=True):
    x_k, z_k, pi_k = generate_init_pt(A, b, c)
    num_constraints, num_vars = A.shape
    I = np.identity(num_vars)
    e = np.ones(num_vars)
    n_iter = 0
    while not met_stopping_condition(Q, A, b, c, x_k, z_k, pi_k, n_iter):
        # Compute affine direction (Predictor step)
        Z_k = np.diag(z_k)
        X_k = np.diag(x_k)
        rp_k = A@x_k - b
        rd_k = -Q.T@x_k + A.T@pi_k + z_k - c
        F = np.block([rd_k, rp_k, X_k@Z_k@e])
        M = np.block([[-Q, A.T,  I],
                      [A, np.zeros((num_constraints, num_constraints + num_vars))],
                      [Z_k,  np.zeros((num_vars, num_constraints)), X_k]
                     ])
        d_affine_x, d_affine_z, d_affine_pi = get_d(M, -F, num_vars, num_constraints)
        alpha_aff_x, alpha_aff_z = get_alpha(Q, x_k, z_k, d_affine_x, d_affine_z)
        
        # Compute u
        y_k = x_k.T@z_k / num_vars
        y_affine_k = (x_k + alpha_aff_x*d_affine_x).T@(z_k + alpha_aff_z*d_affine_z) / num_vars
        tau = (y_affine_k / y_k)**3
        
        # Compute direction (Corrector step)
        Dx_k = np.diag(d_affine_x)
        Dz_k = np.diag(d_affine_z)
        F = np.block([rd_k, rp_k, X_k@Z_k@e + Dx_k@Dz_k@e - tau*y_k*e])
        d_x, d_z, d_pi = get_d(M, -F, num_vars, num_constraints)
        alpha_x, alpha_z =  get_alpha(Q, x_k, z_k, d_x, d_z, 0.95)
        
        # compute points
        x_k = x_k + alpha_x * d_x
        pi_k = pi_k + alpha_z * d_pi
        z_k = z_k + alpha_z * d_z
        
        if verbose:
            print(f"\n-------ITERATION: {n_iter}------")
            print("rp: ", np.round(rp_k,4))
            print("rd: ", np.round(rd_k,4))
            print("d_affine_x: ",np.round(d_affine_x,4), "d_affine_z: ", np.round(d_affine_z,4), "d_affine_pi: ", np.round(d_affine_pi,4))
            print("alpha_aff_x: ", np.round(alpha_aff_x,4), "alpha_aff_z: ", np.round(alpha_aff_z,4))
            print("y: ", np.round(y_k,4), "y_affine: ", np.round(y_affine_k,4), "tau: ", np.round(tau,4))
            print("d_x: ",np.round(d_x,4), "d_z: ", np.round(d_z,4), "d_pi: ",np.round(d_pi,4) )
            print("alpha_x: ", np.round(alpha_x,4), "alpha_z: ",np.round(alpha_z,4) )
            print("x: ",np.round(x_k,4), "pi: ", np.round(pi_k,4), "z",np.round( z_k,4))
        n_iter+=1        
    return x_k, z_k, pi_k

In [6]:
# # LP
# c = np.array([-3,-2,0,0])
# A = np.array([[1,2,1,0], 
#               [2,1,0,1]])
# Q = np.zeros((4,4))
# b = np.array([20, 15])
# x_0 = None 
# pi_0 = None
# z_0 = None

# QP
c = np.array([0,0,0])
A = np.array([[0.1073,0.0737,0.0627], 
              [1,1,1]])
Q = np.array([[0.02778, 0.00387, 0.00021],
             [0.00387, 0.01112, -0.0002],
             [0.00021, -0.0002, 0.00115]])
b = np.array([0.065, 1])
x_0 = np.array([1,1,1]) 
pi_0 = np.array([1,1])
z_0 = np.array([1,1,1])

In [7]:
min_func(Q, A, b, c)


-------ITERATION: 0------
rp:  [0.1787 2.    ]
rd:  [2.0754 2.0589 2.0615]
d_affine_x:  [-1.058  -0.5559 -0.3861] d_affine_z:  [ 0.058  -0.4441 -0.6139] d_affine_pi:  [-16.0716  -0.4405]
alpha_aff_x:  0.9452 alpha_aff_z:  0.9452
y:  1.0 y_affine:  0.1806 tau:  0.0059
d_x:  [-1.0454 -0.6068 -0.3478] d_z:  [ 0.1126 -0.6342 -0.8833] d_pi:  [-23.3367   0.2845]
alpha_x:  0.9087 alpha_z:  0.9087
x:  [0.05   0.4486 0.6839] pi:  [-20.2067   1.2585] z [1.1024 0.4237 0.1973]

-------ITERATION: 1------
rp:  [0.0163 0.1825]
rd:  [0.1894 0.1879 0.1882]
d_affine_x:  [-0.0476 -0.2492  0.1142] d_affine_z:  [-0.0524 -0.1884 -0.2303] d_affine_pi:  [-4.0718  0.2976]
alpha_aff_x:  0.8569 alpha_aff_z:  0.8569
y:  0.1267 y_affine:  0.0238 tau:  0.0066
d_x:  [-0.0352 -0.2997  0.1523] d_z:  [-0.3605 -0.2434 -0.2016] d_pi:  [ 3.4823 -0.2047]
alpha_x:  0.9299 alpha_z:  0.9299
x:  [0.0173 0.1699 0.8256] pi:  [-16.9685   1.0682] z [0.7671 0.1974 0.0099]

-------ITERATION: 2------
rp:  [0.0011 0.0128]
rd:  [0.013

(array([0.02630305, 0.10244401, 0.87125294]),
 array([2.50181226e-08, 5.14145201e-08, 8.73868823e-10]),
 array([0.00724485, 0.00053272]))